**[EN]** English only

FIX no relevant code here change to md file


# Converting the "Ficheiro de alunos"

This notebook processes a file exported from the "Ficheiro de alunos" in CSV format (command separated values), 
converting it in `Kleio` source files or importing directly into a `Timelink` database.

* For detailed information on the contents of the CSV file and the processing methodolgy see [README_ucalumni.md](README_ucalumni.md)
* For information on the `Kleio` format see [README_timelink_db.md](README_timelink_db.md).
* For the structure of the `Timelink` database see [README_timelink_db.md](README_timelink_db.md)


> _First time use of the `notebooks`: follow instructions in the `README.md` file in this directory._

**[EN]**
## Converting from a terminal window

Use the utility `import-alumni.py`, from whithin the repository top level directory `ucprosop`.

### Getting help for the `import-alumni.py` utility

---
**[PT]**
## Converter a partir do terminal


Usar o utilitário `import-alumni.py`, a partir da directoria raiz do repositório `ucprosop`.


### Obtendo ajuda para o utilitário `import-alumni.py`. 

      
  python notebooks/import-alumni.py --help


>>>

      Usage: import-alumni.py [OPTIONS] CSV_FILE DEST_DIR

        Generate kleio source files for alumni records exported from AUC catalog
        (Archeevo)

      Arguments:
        CSV_FILE  path of the csv file exported from Archeevo  [required]
        DEST_DIR  Destination directory for generated files and databases
                  [required]


    ....


    python notebooks/import-alumni.py database/auc/PT-AUC-ELU-UC-AUC-B-001-001.CSV \
            database/sqlite3\
            --db-connection "sqlite:///database/sqlite3/ucalumni.db?check_same_thread=False"
            --rows 0
            --batch 500
            --dryrun

#### Converting from within the notebook

> _NOTE: The `notebook` must be kept open during the convertion, which might take a few hours.


In [2]:
import os
cwd = os.getcwd()
print("Current dir (should end in 'notebooks'):",cwd)

Current dir (should end with 'notebooks'): /Users/jrc/mhk-home/sources/ucprosop/ucprosop/notebooks


In [4]:
from ucalumni.importer import import_auc_alumni
from ucalumni.aluno import Aluno
from ucalumni import extractors
from ucalumni import mapping

catalog = '../database/auc/PT-AUC-ELU-UC-AUC-B-001-001.CSV'
destdir = '../database/sqlite3' # destination directory for database file (sqlite only)
                         # connection string for direct import to database
con_string = f"sqlite:///{destdir}/ucalumni.db?check_same_thread=False"
# This will store the connection string for other notebooks
# To retrieve the in another notebook use %store -r con_string 
%store con_string
rows= 0                 # stop after rows, 0 to process everything
batch = 100            # number of alumni per kleio file
dryrun = False           # if true do not write, just output in notebook
echo   = False           # if true each row in csv file will be printed in notebook


Stored 'con_string' (str)


In [ ]:

import_auc_alumni(  catalog,
                    destdir,
                    con_string,
                    rows,
                    batch,
                    dryrun,
                    echo
                    )

## Gerar ficheiros Kleio para posterior importação

Esta modalidade produz ficheiros na notação `Kleio`.  

Embora exija um posterior passo de importação, tem a vantagem de produzir 
uma representação dos dados num formato legível, de fácil permuta e conservação. 

Os ficheiros produzidos são eficazmente comprimidos com utilitários tipo "Zip".

Executar a célula seguinte converte o ficheiro de alunos para ficheiros `Kleio`, com 500 alunos em cada ficheiro.

Os ficheiros têm de ser posteriormente traduzidos e importados em Timelink ou MHK.



In [ ]:
catalog = '../database/other_exports/ficheiro-alunos-auc/PT-AUC-ELU-UC-AUC-B-001-001.CSV'
destdir = '../sources/' # destination directory for Kleio files
con_string = None         # connection string for direct import to database
rows= 0                 # stop after rows, 0 to process everything
nalumni = 500            # limit of alumni per kleio file
dryrun = False           # if true do not write, just output in notebook
echo   = False           # if true each row in csv file will be printed in notebook
import_auc_alumni(  catalog,
                    destdir,
                    con_string,
                    rows,
                    nalumni,
                    dryrun,
                    echo
                    )

## Check first and last student



In [3]:
from sqlalchemy import select,desc
from sqlalchemy.orm import sessionmaker
from timelink.mhk.models import base
from timelink.mhk.models.person import Person
from timelink.mhk.models.db import TimelinkDB, SQLALCHEMY_ECHO


Session = sessionmaker()

connection_string="sqlite:///../database/sqlite3/ucalumni.db?check_same_thread=False"

SQLALCHEMY_ECHO = True
db = TimelinkDB(connection_string)

Session.configure(bind=db.engine())

stmt = select(Person).order_by(Person.updated).limit(1)
with Session() as session:
    first = session.execute(stmt).scalar()
    print(first.id,first.name,first.updated)

stmt = select(Person).order_by(desc(Person.updated)).limit(1)
with Session() as session:
    last = session.execute(stmt).scalar()
    print(last.id,last.name, last.updated)


140337 António Pinto Abadeço 2022-03-21 07:32:26.613268
196220 Vasco de Oliveira Zuzarte 2022-03-21 17:41:20.281159
